In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!unzip -q "/content/drive/MyDrive/건설용 자갈 분류 DACON/train.zip" -d "/content/gravel_data"

In [ ]:
import os

root_dir = "/content/gravel_data/train"
folders = sorted([f for f in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, f))])
print("클래스 폴더 수:", len(folders))
print("클래스 이름 목록:", folders)

클래스 폴더 수: 7
클래스 이름 목록: ['Andesite', 'Basalt', 'Etc', 'Gneiss', 'Granite', 'Mud_Sandstone', 'Weathered_Rock']


In [ ]:
!pip install timm
!pip install resnest

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 73.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 104.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 62.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 113.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjit

In [ ]:
import os
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset, Subset
from torchvision import transforms
from PIL import Image
from sklearn.model_selection import StratifiedShuffleSplit
from resnest.torch import resnest200
from tqdm import tqdm
from sklearn.metrics import f1_score, accuracy_score

# ⚙️ 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 128
NUM_CLASSES = 7
IMG_SIZE = 224
EPOCHS = 15
LR = 2e-4

# 📁 Dataset 구현
class GravelDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert("RGB")
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

# 🔀 Stratified Split
def stratified_split(image_paths, labels, test_size=0.1):
    sss = StratifiedShuffleSplit(n_splits=1, test_size=test_size, random_state=42)
    for train_idx, val_idx in sss.split(image_paths, labels):
        return train_idx, val_idx

# 🧪 Data Preparation
def prepare_dataloaders(image_paths, labels):
    transform = transforms.Compose([
        transforms.Resize((IMG_SIZE, IMG_SIZE)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])

    train_idx, val_idx = stratified_split(image_paths, labels)
    train_dataset = GravelDataset([image_paths[i] for i in train_idx],
                                  [labels[i] for i in train_idx], transform)
    val_dataset = GravelDataset([image_paths[i] for i in val_idx],
                                [labels[i] for i in val_idx], transform)

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=8, pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=8, pin_memory=True)
    return train_loader, val_loader

# 📦 모델 불러오기
from torchvision.models import resnet101

def load_model():
    model = resnet101(pretrained=True)  # torchvision에서 사전학습된 모델
    model.fc = nn.Linear(model.fc.in_features, NUM_CLASSES)
    return model.to(device)

# 🔧 Loss, Optimizer, Scheduler
def get_training_components(model):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.AdamW(model.parameters(), lr=LR)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)
    return criterion, optimizer, scheduler

from tqdm import tqdm

def train(model, loader, criterion, optimizer):
    model.train()
    running_loss = 0.0
    all_preds = []
    all_labels = []

    progress_bar = tqdm(loader, desc="[Train]", leave=False)
    for images, labels in progress_bar:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        preds = torch.argmax(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

        # 현재 진행률 상태 출력
        progress_bar.set_postfix(loss=loss.item())

    epoch_loss = running_loss / len(loader.dataset)
    epoch_acc = accuracy_score(all_labels, all_preds)
    epoch_f1 = f1_score(all_labels, all_preds, average='macro')
    return epoch_loss, epoch_acc, epoch_f1



def validate(model, loader, criterion):
    model.eval()
    running_loss = 0.0
    all_preds = []
    all_labels = []

    progress_bar = tqdm(loader, desc="[Valid]", leave=False)
    with torch.no_grad():
        for images, labels in progress_bar:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * images.size(0)
            preds = torch.argmax(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

            progress_bar.set_postfix(loss=loss.item())

    epoch_loss = running_loss / len(loader.dataset)
    epoch_acc = accuracy_score(all_labels, all_preds)
    epoch_f1 = f1_score(all_labels, all_preds, average='macro')
    return epoch_loss, epoch_acc, epoch_f1





In [ ]:
import os
import glob

def load_image_paths_and_labels(root_dir, exts=["jpg", "png", "jpeg"]):
    """
    폴더 구조 예시:
    root_dir/
        class0/
            img1.jpg
            img2.jpg
        class1/
            img3.jpg
            img4.jpg
    """
    image_paths = []
    labels = []
    class_names = sorted([
        d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))
    ])

    for idx, class_name in enumerate(class_names):
        class_dir = os.path.join(root_dir, class_name)
        for ext in exts:
            files = glob.glob(os.path.join(class_dir, f"*.{ext}"))
            image_paths.extend(files)
            labels.extend([idx] * len(files))

    return image_paths, labels, class_names


In [ ]:
# 실제 데이터셋 경로 지정 (예: Colab이라면 /content/drive/MyDrive/... )
dataset_root = "/content/gravel_data/train"

image_paths, labels, class_names = load_image_paths_and_labels(dataset_root)

print(f"🔍 총 이미지 수: {len(image_paths)}")
print(f"🏷️ 클래스 목록: {class_names}")
print(f"📊 클래스별 샘플 수: {[labels.count(i) for i in range(len(class_names))]}")

🔍 총 이미지 수: 380020
🏷️ 클래스 목록: ['Andesite', 'Basalt', 'Etc', 'Gneiss', 'Granite', 'Mud_Sandstone', 'Weathered_Rock']
📊 클래스별 샘플 수: [43802, 26810, 15935, 73914, 92923, 89467, 37169]


In [ ]:
# 0. 필요한 패키지 불러오기
import torch
from torch import nn
from sklearn.metrics import accuracy_score, f1_score
from tqdm import tqdm
# + torchvision, DataLoader 관련 import도 필요

# 1. 이미지 경로와 라벨 만들기
dataset_root = "/content/gravel_data/train"
image_paths, labels, class_names = load_image_paths_and_labels(dataset_root)

# 2. DataLoader 준비
train_loader, val_loader = prepare_dataloaders(image_paths, labels)

# 3. 모델 불러오기
model = load_model()

# 4. 학습 구성 요소 준비
criterion, optimizer, scheduler = get_training_components(model)

# 5. 학습 루프 시작 (여기서부터 너가 적은 부분!)
best_f1 = 0.0

for epoch in range(EPOCHS):
    print(f"\nEpoch {epoch+1}/{EPOCHS}")
    train_loss, train_acc, train_f1 = train(model, train_loader, criterion, optimizer)
    val_loss, val_acc, val_f1 = validate(model, val_loader, criterion)
    scheduler.step()

    print(f"[Train] Loss: {train_loss:.4f}, Acc: {train_acc:.4f}, Macro F1: {train_f1:.4f}")
    print(f"[Valid] Loss: {val_loss:.4f}, Acc: {val_acc:.4f}, Macro F1: {val_f1:.4f}")

    if val_f1 > best_f1:
        best_f1 = val_f1
        torch.save(model.state_dict(), "/content/drive/MyDrive/자갈 train 및 test 사진/best_model_resnet200.pth")
        print("✅ Best model saved (Macro F1)!")


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)



Epoch 1/15


[Train] Loss: 0.5143, Acc: 0.8195, Macro F1: 0.7562
[Valid] Loss: 0.4888, Acc: 0.8290, Macro F1: 0.7789
✅ Best model saved (Macro F1)!

Epoch 2/15


[Train] Loss: 0.3847, Acc: 0.8657, Macro F1: 0.8127
[Valid] Loss: 0.4178, Acc: 0.8565, Macro F1: 0.7956
✅ Best model saved (Macro F1)!

Epoch 3/15


[Train] Loss: 0.3191, Acc: 0.8886, Macro F1: 0.8416
[Valid] Loss: 0.4238, Acc: 0.8560, Macro F1: 0.7962
✅ Best model saved (Macro F1)!

Epoch 4/15


[Train] Loss: 0.2594, Acc: 0.9087, Macro F1: 0.8682
[Valid] Loss: 0.4143, Acc: 0.8605, Macro F1: 0.8169
✅ Best model saved (Macro F1)!

Epoch 5/15


[Train] Loss: 0.1965, Acc: 0.9303, Macro F1: 0.8984
[Valid] Loss: 0.3829, Acc: 0.8769, Macro F1: 0.8283
✅ Best model saved (Macro F1)!

Epoch 6/15


[Train] Loss: 0.1301, Acc: 0.9534, Macro F1: 0.9323
[Valid] Loss: 0.4721, Acc: 0.8663, Macro F1: 0.8216

Epoch 7/15


[Train] Loss: 0.0781, Acc: 0.9724, Macro F1: 0.9613
[Valid] Loss: 0.5218, Acc: 0.8737, Macro F1: 0.8250

Epoch 8/15


[Train]:  44%|████▍     | 1177/2673 [19:01<24:08,  1.03it/s, loss=0.0186]